In [33]:
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from imblearn.under_sampling import OneSidedSelection
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import f1_score, accuracy_score, recall_score, precision_score, roc_auc_score
import pandas as pd
import numpy as np


In [5]:
data = pd.read_csv('../datasets/BankChurners.csv')
list = ['Attrition_Flag', 'Total_Trans_Ct', 'Total_Trans_Amt', 'Total_Revolving_Bal', 'Total_Ct_Chng_Q4_Q1', 'Contacts_Count_12_mon', 'Total_Relationship_Count', 'Months_Inactive_12_mon', 'Months_on_book']
data = data[list]

In [6]:
object_columns = data.select_dtypes('object').columns

for i in object_columns:

    lb = LabelEncoder()
    lb.fit(data[i])
    data[i] = lb.transform(data[i])
    
    print(f'category : {np.unique(data[i])}\nclasses : {lb.classes_}\n')

input = data.iloc[:,1:]
target = data.iloc[:,0]

category : [0 1]
classes : ['Attrited Customer' 'Existing Customer']



## XGBClassifier
##### 설명 참고 : https://wooono.tistory.com/97

In [11]:
x_train, x_test, y_train, y_test = train_test_split(input, target, random_state=42, test_size=0.2)

ss = StandardScaler()
x_train = ss.fit_transform(x_train)
x_test = ss.fit_transform(x_test)

oss = OneSidedSelection()
x_train, y_train = oss.fit_resample(x_train, y_train)

xgb = XGBClassifier(random_state=42)

xgb_param_grid = {'n_estimators' : [100, 200],
                'learning_rate' : [0.01, 0.05, 0.1],
                'max_depth' : [3, 5, 7],
                'gamma' : [0, 1, 2]}

xgb_grid = GridSearchCV(xgb, param_grid=xgb_param_grid, scoring='f1', verbose=0, n_jobs=1)
xgb_grid.fit(x_train, y_train)

print(f'best f1 : {xgb_grid.best_score_}')
print('best param : ', xgb_grid.best_params_)

## 참고 : https://cjh34544.tistory.com/m/4
## http://aispiration.com/model/model-python-xgboost-hyper.html

best f1 : 0.9790588684973764
best param :  {'gamma': 1, 'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 200}


## Logistic

In [13]:
x_train, x_test, y_train, y_test = train_test_split(input, target, random_state=42, test_size=0.2)

ss = StandardScaler()
x_train = ss.fit_transform(x_train)
x_test = ss.fit_transform(x_test)

oss = OneSidedSelection()
x_train, y_train = oss.fit_resample(x_train, y_train)

lr = LogisticRegression(random_state=42)

lr_param_grid = {'C' : [0.001, 0.01, 0.1, 1, 10],
                'penalty' : ['l1', 'l2']}

lr_grid = GridSearchCV(lr, param_grid=lr_param_grid, scoring='f1', verbose=0, n_jobs=1)
lr_grid.fit(x_train, y_train)

print(f'best f1 : {lr_grid.best_score_}')
print('best param : ', lr_grid.best_params_)

# 참고 : https://wikidocs.net/16594


best f1 : 0.9432743954066511
best param :  {'C': 0.1, 'penalty': 'l2'}


d:\Python310\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
25 fits failed out of a total of 50.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
25 fits failed with the following error:
Traceback (most recent call last):
  File "d:\Python310\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "d:\Python310\lib\site-packages\sklearn\linear_model\_logistic.py", line 1461, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "d:\Python310\lib\site-packages\sklearn\linear_model\_logistic.py", line 447, in _check_solver
    raise ValueError(
ValueError: Solver lbfgs supports only 'l2' or 'none

## RandomForest

In [14]:
x_train, x_test, y_train, y_test = train_test_split(input, target, random_state=42, test_size=0.2)

ss = StandardScaler()
x_train = ss.fit_transform(x_train)
x_test = ss.fit_transform(x_test)

oss = OneSidedSelection()
x_train, y_train = oss.fit_resample(x_train, y_train)

rf = RandomForestClassifier(random_state=42)

rf_param_grid = {'n_estimators' : [100, 200],
                'max_depth' : [3, 5, 7],
                'min_samples_leaf' : [8, 12, 16],
                'min_samples_split' : [8, 16, 20]}

rf_grid = GridSearchCV(rf, param_grid=rf_param_grid, scoring='f1', verbose=0, n_jobs=1)
rf_grid.fit(x_train, y_train)

print(f'best f1 : {rf_grid.best_score_}')
print('best param : ', rf_grid.best_params_)

## 참고 : https://techblog-history-younghunjo1.tistory.com/102
## https://jaaamj.tistory.com/35


best f1 : 0.967217153494609
best param :  {'max_depth': 7, 'min_samples_leaf': 8, 'min_samples_split': 20, 'n_estimators': 200}


In [16]:
x_train, x_test, y_train, y_test = train_test_split(input, target, random_state=42, test_size=0.2)

ss = StandardScaler()
x_train = ss.fit_transform(x_train)
x_test = ss.fit_transform(x_test)

oss = OneSidedSelection(random_state=42)
x_train, y_train = oss.fit_resample(x_train, y_train)

rf = RandomForestClassifier(random_state=42)

rf_param_grid = {'n_estimators' : [100],
                'min_samples_leaf' : [1],
                'min_samples_split' : [2]}

rf_grid = GridSearchCV(rf, param_grid=rf_param_grid, scoring='f1', verbose=0, n_jobs=1)
rf_grid.fit(x_train, y_train)

print(f'best f1 : {rf_grid.best_score_}')
print('best param : ', rf_grid.best_params_)

## 참고 : https://techblog-history-younghunjo1.tistory.com/102
## https://jaaamj.tistory.com/35


best f1 : 0.9776136754152412
best param :  {'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}


#### CatBoostClassifier는 파라미터 조정이 성능에 크게 영향을 미치지 않는다는 말이 많아 일단 생략함
##### https://velog.io/@jus6886/Catboost
##### https://undeadkwandoll.tistory.com/61
##### https://www.kci.go.kr/kciportal/ci/sereArticleSearch/ciSereArtiView.kci?sereArticleSearchBean.artiId=ART002698429
#### CatBoost 설명
##### https://dailyheumsi.tistory.com/136
##### https://techblog-history-younghunjo1.tistory.com/199
##### https://heeya-stupidbutstudying.tistory.com/43?category=950711


#### lightGBM은 10000개 이하의 데이터에 overfitting하기 쉬워서 사용 x
##### https://nurilee.com/2020/04/03/lightgbm-definition-parameter-tuning/
##### https://mac-user-guide.tistory.com/79

## Voting
##### 코드 참고 : https://eunki.tistory.com/60
##### https://nonmeyet.tistory.com/entry/Python-Voting-Classifiers%EB%8B%A4%EC%88%98%EA%B2%B0-%EB%B6%84%EB%A5%98%EC%9D%98-%EC%A0%95%EC%9D%98%EC%99%80-%EA%B5%AC%ED%98%84


##### 모델 4개 사용한 hard voting

In [38]:
x_train, x_test, y_train, y_test = train_test_split(input, target, random_state=42, test_size=0.2)

ss = StandardScaler()
x_train = ss.fit_transform(x_train)
x_test = ss.fit_transform(x_test)

oss = OneSidedSelection(random_state=42)
x_train, y_train = oss.fit_resample(x_train, y_train)

xgb = XGBClassifier(gamma=1, learning_rate=0.1, max_depth=5, n_estimators=200)
lr = LogisticRegression(C=0.1, penalty='l2')
rf = RandomForestClassifier(verbose=0)
cat = CatBoostClassifier(verbose=0)

model = VotingClassifier(estimators=[('xgb', xgb), ('lr', lr), ('rf', rf), ('cat', cat)], weights=[2, 1, 2, 2], voting='hard')
model.fit(x_train, y_train)
pred = model.predict(x_test)

acc = accuracy_score(y_test, pred)
recall = recall_score(y_test, pred)
precision = precision_score(y_test, pred)
f1 = f1_score(y_test, pred)

print('accuracy : {0} recall : {1}, precision : {2}, f1 : {3}'.format(acc, recall, precision, f1))

accuracy : 0.9639684106614018 recall : 0.9864626250735727, precision : 0.9710312862108922, f1 : 0.9786861313868612


#### 모델 4개 사용한 soft voting

In [39]:
x_train, x_test, y_train, y_test = train_test_split(input, target, random_state=42, test_size=0.2)

ss = StandardScaler()
x_train = ss.fit_transform(x_train)
x_test = ss.fit_transform(x_test)

oss = OneSidedSelection(random_state=42)
x_train, y_train = oss.fit_resample(x_train, y_train)

xgb = XGBClassifier(gamma=1, learning_rate=0.1, max_depth=5, n_estimators=200)
lr = LogisticRegression(C=0.1, penalty='l2')
rf = RandomForestClassifier(verbose=0)
cat = CatBoostClassifier(verbose=0)

model = VotingClassifier(estimators=[('xgb', xgb), ('lr', lr), ('rf', rf), ('cat', cat)], weights=[2, 1, 2, 2], voting='soft')
model.fit(x_train, y_train)
pred = model.predict(x_test)

acc = accuracy_score(y_test, pred)
recall = recall_score(y_test, pred)
precision = precision_score(y_test, pred)
f1 = f1_score(y_test, pred)

print('accuracy : {0} recall : {1}, precision : {2}, f1 : {3}'.format(acc, recall, precision, f1))

accuracy : 0.9615004935834156 recall : 0.987051206592113, precision : 0.9676860934795153, f1 : 0.9772727272727273


#### 모델 3개 사용한 hard voting

In [40]:
x_train, x_test, y_train, y_test = train_test_split(input, target, random_state=42, test_size=0.2)

ss = StandardScaler()
x_train = ss.fit_transform(x_train)
x_test = ss.fit_transform(x_test)

oss = OneSidedSelection(random_state=42)
x_train, y_train = oss.fit_resample(x_train, y_train)

xgb = XGBClassifier(gamma=1, learning_rate=0.1, max_depth=5, n_estimators=200)
lr = LogisticRegression(C=0.1, penalty='l2')
rf = RandomForestClassifier(verbose=0)
cat = CatBoostClassifier(verbose=0)

model = VotingClassifier(estimators=[('xgb', xgb), ('rf', rf), ('cat', cat)], weights=[1, 1, 1], voting='hard')
model.fit(x_train, y_train)
pred = model.predict(x_test)

acc = accuracy_score(y_test, pred)
recall = recall_score(y_test, pred)
precision = precision_score(y_test, pred)
f1 = f1_score(y_test, pred)
roc = roc_auc_score(y_test, pred)

print('accuracy : {0} recall : {1}, precision : {2}, f1 : {3}'.format(acc, recall, precision, f1))

accuracy : 0.9644619940769991 recall : 0.9864626250735727, precision : 0.9715942028985507, f1 : 0.9789719626168224


#### 모델 3개 사용한 soft voting

In [41]:
x_train, x_test, y_train, y_test = train_test_split(input, target, random_state=42, test_size=0.2)

ss = StandardScaler()
x_train = ss.fit_transform(x_train)
x_test = ss.fit_transform(x_test)

oss = OneSidedSelection(random_state=42)
x_train, y_train = oss.fit_resample(x_train, y_train)

xgb = XGBClassifier(gamma=1, learning_rate=0.1, max_depth=5, n_estimators=200)
lr = LogisticRegression(C=0.1, penalty='l2')
rf = RandomForestClassifier(verbose=0)
cat = CatBoostClassifier(verbose=0)

model = VotingClassifier(estimators=[('xgb', xgb), ('rf', rf), ('cat', cat)], weights=[1, 1, 1], voting='soft')
model.fit(x_train, y_train)
pred = model.predict(x_test)

acc = accuracy_score(y_test, pred)
recall = recall_score(y_test, pred)
precision = precision_score(y_test, pred)
f1 = f1_score(y_test, pred)
roc = roc_auc_score(y_test, pred)

print('accuracy : {0} recall : {1}, precision : {2}, f1 : {3}'.format(acc, recall, precision, f1))

accuracy : 0.9649555774925962 recall : 0.985285462036492, precision : 0.9732558139534884, f1 : 0.9792336940625913


#### 평가지표로 f1 score를 쓰는 이유
##### https://towardsdatascience.com/read-this-before-using-roc-auc-as-a-metric-c84c2d5af621
##### https://stackoverflow.com/questions/44172162/f1-score-vs-roc-auc
##### https://neptune.ai/blog/f1-score-accuracy-roc-auc-pr-auc

#### Boosting Model 비교
##### https://medium.com/@divyagera2402/boosting-algorithms-adaboost-gradient-boosting-xgb-light-gbm-and-catboost-e7d2dbc4e4ca
##### http://dmqm.korea.ac.kr/activity/seminar/323


#### 인코딩 방법
##### https://wyatt37.tistory.com/11

#### 한 것요약
##### 원핫인코딩은 차원을 늘려 과적합되기 쉽다. 물론 규제로 어느정도 커버할 수 있으나 트리계열에서는 해당 변수가 아예 제외되는 문제점을 가진다
##### catboost는 파라미터 수정을 하지 않아도 효과가 나쁘지 않다
##### lightgbm은 데이터의 수가 너무 적어 사용할 수 없다
##### ROC가 F1보다 불균형 데이터 셋에 대해 관대한? 경향이 있어서 불균형 데이터 셋에는 F1을 평가지표로 사용한다.
